In [6]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

# This ensures Plotly output works in multiple places:
# In VSCode and also nbconvert from jupyter notebook to HTML
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "notebook_connected"


In [7]:
# Customize Plot
bnw = go.layout.Template(
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            showline=True,
            linecolor="black",
            linewidth=2,
            mirror=True,
            title=""
        ),
        margin=go.layout.Margin(
            l=70,
            r=30
        ),
        yaxis=go.layout.YAxis(
            showline=True,
            linecolor="black",
            linewidth=2,
            mirror=True,
            title=""
        ),
        font=go.layout.Font(
            # family="Old Standard TT",
            size=12
        ),
        title=go.layout.Title(
            font=go.layout.title.Font(
                # family="Old Standard TT",
                size=18
            )
        ),
        # paper_bgcolor="#eaeaf2"
#         font=dict(color="white"),
#         plot_bgcolor="black"
        paper_bgcolor="#D0E3F1",
#         font=dict(color="white")
    ),
)

pio.templates["bnw"] = bnw

px.defaults.template = "seaborn+bnw"

In [8]:
# Functions: 
def clean_notebook():
    return 'Hi'

def data_checker(df: pd.Series, check_unique=False, check_null=True):
    if check_unique:
        print("Unique values", df.unique())
    
    if check_null:
        print("Number of NA Values:", int(df.isnull().sum()))
    
    

In [9]:
def create_figure_with_dropdown(df_new, 
                                options=["What is your gender?", "Are you a domestic or international student?"],
                                unique_values_for_each_option=[
                                    ["Male",  "Female"],
                                    ["International", "Domestic", "Did not wish to disclose"]
                                ]):
    fig = go.Figure()

    # Add initial traces
    length_per_option = [len(unique_values_for_one_option) for unique_values_for_one_option in unique_values_for_each_option]
    
    # Generate Traces
    for i, unique_values_for_one_option in enumerate(unique_values_for_each_option):
        for unique_value in unique_values_for_one_option:
            # Have to figure out how to modularize this
            fig.add_trace(go.Histogram(x=list(df_new.loc[df_new[options[i]] == unique_value, options[i]]), 
                                    name=unique_value,
                                    visible=True if i == 0 else False,
                                    marker_autocolorscale=True))
    
    visibilities = []
    # i is the option being analyzed
    for i, _ in enumerate(options):        
        visibility = []
        # j are the lengths for each option
        for j, option_length in enumerate(length_per_option):
            arr = [True] * option_length if i == j else [False] * option_length
            visibility.extend(arr)
        
        visibilities.append(visibility)

    # px.histogram(df, x="What is your gender?", color="What is your gender?")
    fig.update_traces(hovertemplate="%{x}=%{y}", marker_autocolorscale=True)
    fig.update_layout(
        updatemenus=[
            dict(
                active=0,
                buttons=list([
                    dict(label=option_name,
                         method="update",
                         args=[
                            {"visible" : visibilities[i]},
                            {"title" : option_name} 
                         ])
                for i, option_name in enumerate(options)])
                    
                    # dict(label="What is your gender?",
                    #     method="update",
                    #     args=[
                    #         {
                    #             "visible" : [True, False]
                    #         },
                    #         {
                    #             "title" : "What is your gender?"
                    #         },
                    #     ],
                    # ),
                    # dict(
                    #     label="Are you a domestic or international student?",
                    #     method="update",
                    #     args=[
                    #         {
                    #             "visible" : [False, True]
                                
                    #         },
                    #         {
                    #             "title" : "Are you a domestic or international student?",
                    #         },   
                    #     ],
                    # )
            )
        ]
    )

    fig.update_layout(title_text=options[0])

    return fig

In [11]:
def remove_redundancy(series: pd.Series):
    series_remove_redundancy = series.str.split(', ').map(lambda x: ", ".join(sorted(x)))
    return series_remove_redundancy
    